In [ ]:
!pip install librosa tensorflowjs

     |████████████████████████████████| 71kB 3.3MB/s 
     |████████████████████████████████| 112kB 6.8MB/s 
  Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0


In [ ]:

import glob
import json
import os
import random
import pandas as pd


import librosa
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy.io import wavfile
import tensorflow as tf
import tensorflowjs as tfjs
import tqdm

print(tf.__version__)
print(tfjs.__version__)

2.4.1
3.3.0


In [ ]:
# Download the TensorFlow.js Speech Commands model and the associated
# preprocesssing model.
!mkdir -p /tmp/tfjs-sc-model
!curl -o /tmp/tfjs-sc-model/metadata.json -fsSL https://storage.googleapis.com/tfjs-models/tfjs/speech-commands/v0.3/browser_fft/18w/metadata.json
!curl -o /tmp/tfjs-sc-model/model.json -fsSL https://storage.googleapis.com/tfjs-models/tfjs/speech-commands/v0.3/browser_fft/18w/model.json
!curl -o /tmp/tfjs-sc-model/group1-shard1of2 -fSsL https://storage.googleapis.com/tfjs-models/tfjs/speech-commands/v0.3/browser_fft/18w/group1-shard1of2
!curl -o /tmp/tfjs-sc-model/group1-shard2of2 -fsSL https://storage.googleapis.com/tfjs-models/tfjs/speech-commands/v0.3/browser_fft/18w/group1-shard2of2
!curl -o /tmp/tfjs-sc-model/sc_preproc_model.tar.gz -fSsL https://storage.googleapis.com/tfjs-models/tfjs/speech-commands/conversion/sc_preproc_model.tar.gz
!cd /tmp/tfjs-sc-model/ && tar xzvf sc_preproc_model.tar.gz

sc_preproc_model/
sc_preproc_model/assets/
sc_preproc_model/variables/
sc_preproc_model/variables/variables.data-00000-of-00001
sc_preproc_model/variables/variables.index
sc_preproc_model/saved_model.pb


In [ ]:
# Download Speech Commands v0.02 dataset. The dataset contains 30+ word and
# sound categories, but we will only use a subset of them

!mkdir -p /tmp/speech_commands_v0.02
!curl -o /tmp/speech_commands_v0.02/speech_commands_v0.02.tar.gz -fSsL http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz
!cd  /tmp/speech_commands_v0.02 && tar xzf speech_commands_v0.02.tar.gz

In [ ]:
# Load the preprocessing model, which transforms audio waveform into 
# spectrograms (2D image-like representation of sound).
# This preprocessing model replicates WebAudio's AnalyzerNode.getFloatFrequencyData
# (https://developer.mozilla.org/en-US/docs/Web/API/AnalyserNode/getFloatFrequencyData).
# It performs short-time Fourier transform (STFT) using a length-2048 Blackman
# window. It opeartes on mono audio at the 44100-Hz sample rate.

preproc_model_path = '/tmp/tfjs-sc-model/sc_preproc_model'
preproc_model = tf.keras.models.load_model(preproc_model_path)
preproc_model.summary()
preproc_model.input_shape

Model: "audio_preproc"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
audio_preprocessing_layer (A (None, None, 232, 1)      2048      
Total params: 2,048
Trainable params: 0
Non-trainable params: 2,048
_________________________________________________________________


(None, 44032)

In [ ]:
# COUGHVID DATASET
!wget https://zenodo.org/record/4048312/files/public_dataset.zip?download=1
!mv public_dataset.zip?download=1 public_dataset.zip
!unzip -q public_dataset.zip
!mkdir 'custom_dataset'

--2021-03-19 02:50:00--  https://zenodo.org/record/4048312/files/public_dataset.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 951442487 (907M) [application/octet-stream]
Saving to: ‘public_dataset.zip?download=1’

public_dataset.zip? 100%[===================>] 907.37M  14.6MB/s    in 64s     

2021-03-19 02:51:06 (14.2 MB/s) - ‘public_dataset.zip?download=1’ saved [951442487/951442487]



In [ ]:
coughvid  = '/content/public_dataset/'

In [ ]:
VidData   = pd.read_csv(os.path.join(coughvid,'metadata_compiled.csv'),header=0)
# VidData.head()

In [ ]:
labels = VidData['status'].unique()
print(labels)

['healthy' nan 'COVID-19' 'symptomatic']


In [ ]:
VidData   = VidData.loc[VidData['cough_detected'] >= 0.9][['uuid','fever_muscle_pain','respiratory_condition','status']]
VidData.dropna(inplace=True)

extradata = VidData.loc[VidData['status']=='COVID-19']
notradata = VidData.loc[VidData['status']!='COVID-19'][0:1000]

TotData   = pd.concat([extradata,notradata],ignore_index= True)
TotData['DIR'] = coughvid + TotData['uuid'] + '.webm'
TotData['DataSet'] = 'coughvid'
TotData['fever_muscle_pain']    = TotData['fever_muscle_pain'].apply(int)
TotData['respiratory_condition']= TotData['respiratory_condition'].apply(int)
TotData   = pd.concat([TotData.rename(columns={'uuid':'ID','status':'STATUS','fever_muscle_pain':'Fever/MP','respiratory_condition':'ORC'})])
TotData   = TotData.sample(frac=1).reset_index(drop=True)
# TotData.head()

In [ ]:
labels = TotData['STATUS'].unique()
print(labels, len(TotData))

['healthy' 'COVID-19' 'symptomatic'] 1441


In [ ]:
# copy background noise to our new wav_dataset folder which wil contain separate directories for each class
!mkdir /content/wav_dataset
!cp "/tmp/speech_commands_v0.02/_background_noise_" -r /content/wav_dataset/_background_noise_

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# copy over subset of wav data that amil prepared
!cp '/content/drive/MyDrive/coughvid_data2.zip' -r /content

In [ ]:
# unzip coughvid_data2 into /content/coughvid_data2
!unzip -q coughvid_data2.zip

In [ ]:
# create class directories under wav_dataset for healhty and covid-19
!mkdir '/content/wav_dataset/healthy'
!mkdir '/content/wav_dataset/COVID-19'

In [ ]:
healthy_data = []
covid_19_data = []  # df[df['B']==3]['A']
healthy_data.append(TotData[TotData['STATUS'] == "healthy"]['DIR'].values)
covid_19_data.append(TotData[TotData['STATUS'] == "COVID-19"]['DIR'].values)

In [ ]:
# print(healthy_data) #837 healthy, 441 covid
[healthy_data] = healthy_data
healthy_data = healthy_data.tolist()
print(healthy_data, type(healthy_data), len(healthy_data))

[covid_19_data] = covid_19_data
covid_19_data = covid_19_data.tolist()
print(covid_19_data, type(covid_19_data), len(covid_19_data))

In [ ]:
# clean file names in these lists

healthy_data_2 = []
covid_19_data_2 = []

for i in healthy_data:
  path = i.replace("/content/public_dataset/", "")
  healthy_data_2.append(path)
for i in covid_19_data:
  path = i.replace("/content/public_dataset/", "")
  covid_19_data_2.append(path)

print(healthy_data_2)
print(covid_19_data_2)

In [ ]:
import shutil, os

# copy corresponding files into separate healthy or covid-19 directories under /content/
for f in healthy_data_2:
  name = os.path.splitext(f)[0] #get file name without extension
  if name+".wav" in os.listdir('/content/coughvid_data2'):
    # copy to /content/wav_dataset/healthy
    shutil.copy(os.path.join('/content/coughvid_data2/', name + ".wav"), '/content/wav_dataset/healthy')

for f in covid_19_data_2:
  name = os.path.splitext(f)[0]
  if name+".wav" in os.listdir('/content/coughvid_data2'):
    shutil.copy(os.path.join('/content/coughvid_data2/', name + ".wav"), '/content/wav_dataset/COVID-19')

In [ ]:
# Create some constants to be used later.

# Target sampling rate. It is required by the audio preprocessing model.
TARGET_SAMPLE_RATE = 44100
# The specific audio tensor length expected by the preprocessing model.
EXPECTED_WAVEFORM_LEN = preproc_model.input_shape[-1]

# Where the Speech Commands v0.02 dataset has been downloaded.
DATA_ROOT = "/content/wav_dataset" # "/tmp/speech_commands_v0.02"

WORDS = ("_background_noise_snippets_", "healthy", "COVID-19") # ("_background_noise_snippets_", "no", "yes")

In [ ]:
# Unlike word examples, the noise samples in the Speech Commands v0.02 dataset
# are not divided into 1-second snippets. Instead, they are stored as longer
# recordings. Therefore we need to cut them up in to 1-second snippet .wav
# files.

noise_wav_paths = glob.glob(os.path.join(DATA_ROOT, "_background_noise_", "*.wav"))
snippets_dir = os.path.join(DATA_ROOT, "_background_noise_snippets_")
os.makedirs(snippets_dir, exist_ok=True)


def extract_snippets(wav_path, snippet_duration_sec=1.0):
  basename = os.path.basename(os.path.splitext(wav_path)[0])
  sample_rate, xs = wavfile.read(wav_path)
  assert xs.dtype == np.int16
  n_samples_per_snippet = int(snippet_duration_sec * sample_rate)
  i = 0
  while i + n_samples_per_snippet < len(xs):
    snippet_wav_path = os.path.join(snippets_dir, "%s_%.5d.wav" % (basename, i))
    snippet = xs[i : i + n_samples_per_snippet].astype(np.int16)
    wavfile.write(snippet_wav_path, sample_rate, snippet)
    i += n_samples_per_snippet

for noise_wav_path in noise_wav_paths:
  print("Extracting snippets from %s..." % noise_wav_path)
  extract_snippets(noise_wav_path, snippet_duration_sec=1.0)

In [ ]:

def resample_wavs(dir_path, target_sample_rate=44100):
  """Resample the .wav files in an input directory to given sampling rate.
  
  The resampled waveforms are written to .wav files in the same directory with
  file names that ends in "_44100hz.wav".

  44100 Hz is the sample rate required by the preprocessing model. It is also
  the most widely supported sample rate among web browsers and mobile devices.
  For example, see:
  https://developer.mozilla.org/en-US/docs/Web/API/AudioContextOptions/sampleRate
  https://developer.android.com/ndk/guides/audio/sampling-audio

  Args:
    dir_path: Path to a directory that contains .wav files.
    target_sapmle_rate: Target sampling rate in Hz.
  """
  wav_paths = glob.glob(os.path.join(dir_path, "*.wav"))
  resampled_suffix = "_%shz.wav" % target_sample_rate
  for i, wav_path in tqdm.tqdm(enumerate(wav_paths)):
    if wav_path.endswith(resampled_suffix):
      continue
    sample_rate, xs = wavfile.read(wav_path)
    xs = xs.astype(np.float32)
    xs = librosa.resample(xs, sample_rate, TARGET_SAMPLE_RATE).astype(np.int16)
    resampled_path = os.path.splitext(wav_path)[0] + resampled_suffix
    wavfile.write(resampled_path, target_sample_rate, xs)


for word in WORDS:
  word_dir = os.path.join(DATA_ROOT, word)
  assert os.path.isdir(word_dir)
  resample_wavs(word_dir, target_sample_rate=TARGET_SAMPLE_RATE)

In [ ]:
@tf.function
def read_wav(filepath):
  file_contents = tf.io.read_file(filepath)
  return tf.expand_dims(tf.squeeze(tf.audio.decode_wav(
      file_contents, 
      desired_channels=-1,
      desired_samples=TARGET_SAMPLE_RATE).audio, axis=-1), 0)


@tf.function
def filter_by_waveform_length(waveform, label):
  return tf.size(waveform) > EXPECTED_WAVEFORM_LEN


@tf.function
def crop_and_convert_to_spectrogram(waveform, label):
  cropped = tf.slice(waveform, begin=[0, 0], size=[1, EXPECTED_WAVEFORM_LEN])
  return tf.squeeze(preproc_model(cropped), axis=0), label


@tf.function
def spectrogram_elements_finite(spectrogram, label):
  return tf.math.reduce_all(tf.math.is_finite(spectrogram))


def get_dataset(input_wav_paths, labels):
  """Get a tf.data.Dataset given input .wav files and their labels.

  The returned dataset emits 2-tuples of `(spectrogram, label)`, wherein
  - `spectrogram` is a tensor of dtype tf.float32 and shape [43, 232, 1].
    It is z-normalized (i.e., have a mean of ~0.0 and variance of ~1.0).
  - `label` is a tensor of dtype tf.int32 and shape [] (scalar).
  
  Args:
    input_wav_paths: Input audio .wav file paths as a list of string.
    labels: integer labels (class indices) of the input .wav files. Must have
      the same lengh as `input_wav_paths`.

  Returns:
    A tf.data.Dataset object as described above.
  """
  ds = tf.data.Dataset.from_tensor_slices(input_wav_paths)
  # Read audio waveform from the .wav files.
  ds = ds.map(read_wav)
  ds = tf.data.Dataset.zip((ds, tf.data.Dataset.from_tensor_slices(labels)))
  # Keep only the waveforms longer than `EXPECTED_WAVEFORM_LEN`.
  ds = ds.filter(filter_by_waveform_length)
  # Crop the waveforms to `EXPECTED_WAVEFORM_LEN` and convert them to
  # spectrograms using the preprocessing layer.
  ds = ds.map(crop_and_convert_to_spectrogram)
  # Discard examples that contain infinite or NaN elements.
  ds = ds.filter(spectrogram_elements_finite)
  return ds

In [ ]:
input_wav_paths_and_labels = []
for i, word in enumerate(WORDS):
  wav_paths = glob.glob(os.path.join(DATA_ROOT, word, "*_%shz.wav" % TARGET_SAMPLE_RATE))
  print("Found %d examples for class %s" % (len(wav_paths), word))
  labels = [i] * len(wav_paths)
  input_wav_paths_and_labels.extend(zip(wav_paths, labels))
random.shuffle(input_wav_paths_and_labels)

print("input_wav_paths_and_labels: ", input_wav_paths_and_labels)
  
input_wav_paths, labels = ([t[0] for t in input_wav_paths_and_labels],
                           [t[1] for t in input_wav_paths_and_labels])
dataset = get_dataset(input_wav_paths, labels)

# Show some example spectrograms for inspection.
fig = plt.figure(figsize=(40, 100))
dataset_iter = iter(dataset)
num_spectrograms_to_show = 10
for i in range(num_spectrograms_to_show):
  ax = fig.add_subplot(1, num_spectrograms_to_show, i + 1)
  # print(dataset_iter)
  spectrogram, label = next(dataset_iter)
  spectrogram = spectrogram.numpy()
  label = label.numpy()
  plt.imshow(np.flipud(np.squeeze(spectrogram, -1).T), aspect=0.2)
  ax.set_title("Example of \"%s\"" % WORDS[label])
  ax.set_xlabel("Time frame #")
  if i == 0:
    ax.set_ylabel("Frequency bin #")

In [ ]:
# The amount of data we have is relatively small. It fits into typical host RAM
# or GPU memory. For better training performance, we preload the data and
# put it into numpy arrays:
# - xs: The audio features (normalized spectrograms).
# - ys: The labels (class indices).
print(
    "Loading dataset and converting data to numpy arrays. "
    "This may take a few minutes...")
xs = []
ys = []
print(EXPECTED_WAVEFORM_LEN)
iterator = dataset.__iter__()
for i in range(len(labels)):
  # next_element = iterator.get_next()
  try: 
    next_element = iterator.get_next_as_optional()
    xs.append(next_element.get_value()[0].numpy())
    ys.append(next_element.get_value()[1].numpy())
  except:
    continue
    # print(next_element) 
  #   print(next_element.get_value()[0].numpy().shape)
  #   print(next_element.get_value()[1].numpy().shape)

  # print(next_element)
  # print(next_element.has_value())

xs = np.array(xs)
ys = np.array(ys)
# next_element = iterator.get_next()
# xs_and_ys = list(dataset.as_numpy_iterator())
# xs = np.stack([item[0] for item in xs_and_ys])
# ys = np.stack([item[1] for item in xs_and_ys])

# # MY ATTEMPT TO CONVERT DATA TO NP ARRAYS
# # xs should be array of spectrograms, each of size (43, 232, 1)
# # ys should be array of labels
# # sample_rate, xs = wavfile.read(wav_path)
# xs = []
# for i in input_wav_paths:
#   _, x = wavfile.read(i)
# # change x to spectrogram
#   xs.append(x)

# ys = []
# for y in labels:
#   ys.append(y)

# xs = np.array(xs)
# ys = np.array(ys).astype('float32')

print("Done.")

In [ ]:
dataset_shuffle = dataset.shuffle(int(0.3*533), reshuffle_each_iteration=True)
xs_eval = []
ys_eval = []
iterator = dataset_shuffle.__iter__()
for i in range(int(0.3*533)):
  # next_element = iterator.get_next()
  try: 
    next_element = iterator.get_next_as_optional()
    xs_eval.append(next_element.get_value()[0].numpy())
    ys_eval.append(next_element.get_value()[1].numpy())
  except:
    continue
xs_eval = np.array(xs_eval)
ys_eval = np.array(ys_eval)

In [ ]:

tfjs_model_json_path = '/tmp/tfjs-sc-model/model.json'

# Load the Speech Commands model. Weights are loaded along with the topology,
# since we train the model from scratch. Instead, we will perform transfer
# learning based on the model.
orig_model = tfjs.converters.load_keras_model(tfjs_model_json_path, load_weights=True)

# Remove the top Dense layer and add a new Dense layer of which the output
# size fits the number of sound classes we care about.
model_coughvid = tf.keras.Sequential(name="TransferLearnedModel")
for layer in orig_model.layers[:-1]:
  model_coughvid.add(layer)
model_coughvid.add(tf.keras.layers.Dense(units=len(WORDS), activation="softmax"))

# Freeze all but the last layer of the model. The last layer will be fine-tuned
# during transfer learning.
for layer in model_coughvid.layers[:-1]:
  layer.trainable = False

model_coughvid.compile(optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["acc"])
model_coughvid.summary()

In [ ]:
# THESE ARE INPUTS/OUTPUTS THAT MODEL_COUGHVID EXPECTS: 
[print(i.shape, i.dtype) for i in model_coughvid.inputs]
[print(o.shape, o.dtype) for o in model_coughvid.outputs]

In [ ]:
# Train the model.
! pip install --upgrade wandb
import wandb
from wandb.keras import WandbCallback

wandb.init(project="virufy_data", entity="virufy-team", name = "speech_commands")

model_coughvid.fit(xs, ys, batch_size=256, validation_split=0.3, shuffle=True, epochs=50, callbacks=[WandbCallback()])

wandb.finish()

In [ ]:
score = model_coughvid.evaluate(xs_eval, ys_eval)
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

In [ ]:
# Convert the model to TensorFlow.js Layers model format.

tfjs_model_dir = "/tmp/tfjs-model-coughvid"
tfjs.converters.save_keras_model(model_coughvid, tfjs_model_dir)

# Create the metadata.json file.
metadata = {"words": ["_background_noise_"], "frameSize": model_coughvid.input_shape[-2]}
metadata["words"] += WORDS[1:]
with open(os.path.join(tfjs_model_dir, "metadata.json"), "w") as f:
  json.dump(metadata, f)

!ls -lh /tmp/tfjs-model-coughvid

In [ ]:
# from keras.initializers import glorot_uniform
# #Reading the model from JSON file
# with open('/tmp/tfjs-model-coughvid/model.json', 'r') as json_file:
#     json_savedModel= json_file.read()
# #load the model architecture 
# model_j = tf.keras.models.model_from_json(json_savedModel)
# model_j.summary()



# from tensorflow import keras
# # got weights and model from model_coughvid
# weights = model_coughvid.get_weights()
# config = model_coughvid.to_json()
# loaded_model = tf.keras.models.model_from_json(config)

# # load weights to model
# loaded_model.set_weights(weights)

# loaded_model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["acc"])

# score = loaded_model.evaluate(xs_eval, ys_eval)
# print('Test loss:', score[0]) 
# print('Test accuracy:', score[1])

In [ ]:

# Convert the model to TFLite. 

# We need to combine the preprocessing model and the newly trained 3-class model
# so that the resultant model will be able to preform STFT and spectrogram 
# calculation on mobile devices (i.e., without web browser's WebAudio).

combined_model = tf.keras.Sequential(name='CombinedModel')
combined_model.add(preproc_model)
combined_model.add(model_coughvid)
combined_model.build([None, EXPECTED_WAVEFORM_LEN])
combined_model.summary()

# tfjson_output_path = '/tmp/tfjs-sc-model/combined_model.json'
# tfjs.converters.save_keras_model(combined_model, tfjson_output_path)
# converter = combined_model.to_json()
# loaded_model = tf.keras.models.model_from_json(config)

# with open(tfjson_output_path, 'wb') as f:
#     f.write(converter.convert())
# print("Saved tfjson file at: %s" % tflite_output_path)

tflite_output_path = '/tmp/tfjs-sc-model/combined_model.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(combined_model)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS
]
with open(tflite_output_path, 'wb') as f:
    f.write(converter.convert())
print("Saved tflite file at: %s" % tflite_output_path)

In [ ]:
# ! pip install -q tf-nightly
# ! pip install -q tensorflow-model-optimization

# import tensorflow_model_optimization as tfmot

# quantize_model = tfmot.quantization.keras.quantize_model

# # q_aware stands for for quantization aware.
# q_aware_model = quantize_model(model_coughvid)

# # `quantize_model` requires a recompile.
# q_aware_model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["acc"])

# q_aware_model.summary()

In [ ]:
# wandb.init(project="virufy_data", entity="virufy-team", name = "speech_commands_quantized")

# q_aware_model.fit(xs, ys, batch_size=256, validation_split=0.3, shuffle=True, epochs=50, callbacks=[WandbCallback()])

# wandb.finish()

In [ ]:
# # Convert the model to TensorFlow.js Layers model format.

# tfjs_model_dir = "/tmp/tfjs-model-coughvid-quant"
# tfjs.converters.save_keras_model(q_aware_model, tfjs_model_dir)

# # Create the metadata.json file.
# metadata = {"words": ["_background_noise_"], "frameSize": q_aware_model.input_shape[-2]}
# metadata["words"] += WORDS[1:]
# with open(os.path.join(tfjs_model_dir, "metadata.json"), "w") as f:
#   json.dump(metadata, f)

# !ls -lh /tmp/tfjs-model-coughvid-quant

In [ ]:
# quant_file = "/tmp/tfjs-model-coughvid-quant/model.json"
# # with open(quant_file, 'wb') as f:
# #   f.write(tfjs.converters.save_keras_model(q_aware_model, "/tmp/tfjs-model-coughvid-quant"))

# float_file = "/tmp/tfjs-model-coughvid/model.json"
# # with open(float_file, 'wb') as f:
# #   f.write(tfjs.converters.save_keras_model(model_coughvid, "/tmp/tfjs-model-coughvid"))

# print("Float model in Mb:", os.path.getsize(float_file) / float(2**20))
# print("Quantized model in Mb:", os.path.getsize(quant_file) / float(2**20))